In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import savReaderWriter as sav
import pandas as pd
import numpy as np
from datetime import datetime
%matplotlib inline

import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.core.display import display, HTML
import seaborn as sns
import pandas as pd
import numpy as np

# Legend
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import unidecode
plt.rcParams.update({'font.size': 16})

/var/folders/r4/lwrzx5hs7d3cxk4p5zsy7zqh0000gp/T/ipykernel_29032/2461225152.py:13: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [7]:
# Importing data
with open( r'../_data/data_endes.pickle', 'rb') as f:
    df = pickle.load(f )

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/ar8787/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/r4/lwrzx5hs7d3cxk4p5zsy7zqh0000gp/T/ipykernel_29032/441731810.py", line 3, in <module>
    df = pickle.load(f )
         ^^^^^^^^^^^^^^^
  File "/Users/ar8787/anaconda3/lib/python3.11/site-packages/pandas/core/internals/blocks.py", line 2400, in new_block
TypeError: Argument 'placement' has incorrect type (expected pandas._libs.internals.BlockPlacement, got slice)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ar8787/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ar8787/anaconda3/lib/python3.11/site-packages/IPython/core

Check Unique ID

In [ ]:
(df.CASEID + df.year.astype( str )).is_unique

From CMC to datetime. Info about datetime use [this link](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime)

In [ ]:
from datetime import datetime

datetime.strptime( f'09-02-2019', '%m-%d-%Y')

See [this article](https://demographicestimation.iussp.org/content/dhs-century-month-codes) to convert CMC to datetime

In [ ]:
def cmc_date( cmc ):
    year = 1900 + int( ( cmc - 1 ) / 12 )
    month = int( cmc - 12*( year - 1900 ) )
    datetime_object = datetime.strptime( f'{month}/{year}', '%m/%Y')
    
    return datetime_object

In [ ]:
df[ 'inerview_date' ] = df.V008.apply( lambda x: cmc_date( x ) )

In [ ]:
df[ 'inerview_month' ] = df[ 'inerview_date' ].dt.strftime( "%b" )
df[ 'inerview_month1' ] = df[ 'inerview_date' ].dt.month
df[ 'inerview_year' ] = df[ 'inerview_date' ].dt.year
df.sort_values( 'inerview_month1', inplace = True )

In [ ]:
# Settin

fig1 = sns.catplot( data = df, x = 'inerview_month', kind = 'count',
            hue = 'inerview_year', 
            palette = sns.color_palette("mako_r", 5 ), 
            legend = False )


fig1.fig.set_size_inches( 15, 8 )

plt.ylabel( "Number of Interviews" )
plt.xlabel( "Month" )
plt.legend( title = 'Year', bbox_to_anchor = (1.15, 1 ), loc = 0 )

In [ ]:
# Settin
fig1 = sns.catplot( data = df, x = 'inerview_month', kind = 'count',
            col = 'inerview_year', col_wrap = 2,
            legend = False,  color = (0.2, 0.4, 0.6, 0.6) )

# Fix figure size
fig1.fig.set_size_inches( 18, 14 )

# Fix title 
# add space between plots and title
fig1.fig.subplots_adjust(top=0.9)
fig1.fig.suptitle( "Number of Interview by Month and Year", 
                size = 18  )

# Add vertical space between plots
plt.subplots_adjust( hspace = 0.2 )

# Set axis labels
fig1.set_axis_labels( "Month", "Number of Interviews", size = 15 )

# Change title for each subplot
for year in fig1.axes_dict.keys():
    fig1.axes_dict[ year ].set_title( f'Interview Year - { year }' )
    


Distribution of Age

In [ ]:
fig2 = sns.displot( data = df, x = 'V012', hue = 'year', 
            palette = sns.color_palette("mako_r", 5 ), 
           kind = "kde" )


# Make title
fig2.fig.subplots_adjust(top=0.9)
fig2.fig.suptitle( "Distribution of Interviewed Age by Surveys Year", 
                size = 14 )

# Change legend title
fig2._legend.set_title( "Year" )

# Set axis labels
fig2.set_axis_labels( "Age", "Density", size = 15 );


Distribution Total children ever born by place of residence

In [ ]:
df.V102.unique()

In [ ]:
df[ 'urban' ] = df.V102.copy()
df[ 'urban' ].replace( 'Urbana', 'Urbano', inplace = True )
df[ 'urban' ].replace( 'Urban', 'Urbano', inplace = True )

In [ ]:
df.attrs['var_labels']

In [ ]:
# Plot ploints
fig = plt.figure( figsize = ( 10, 9 ) )
sns.stripplot( y = "V201", hue = "year", 
                x = "urban", 
                data = df, jitter = True,
                palette = sns.color_palette("rocket", 5 )[::-1], 
                dodge = True, 
                linewidth = 1, edgecolor = 'white' )

# Plot boxplot
yx = sns.boxplot( y = "V201", hue = "year", 
                x = "urban", 
                data = df, 
                palette = sns.color_palette("rocket", 5 )[::-1], 
                fliersize = 0 )

# Number of labels
yr = df.year.unique().size

# Get legend labels and handles
handles, labels = yx.get_legend_handles_labels()

# When creating the legend, only use the first 5 elements
# to effectively remove the last.
plt.legend(handles[0:yr], labels[0:yr], bbox_to_anchor=(1.05, 1), 
           loc=2, borderaxespad=0., title = "Year" )

# x label
plt.xlabel( "Type of place of residence" )
plt.ylabel( "Total children ever born" );

### Total children by cohort

In [ ]:
sns.histplot( data = df, x = 'V010')
plt.xlabel( "Respondent's year of birth" )

In [ ]:
sns.boxplot( data = df, y = 'V010')
plt.ylabel( "Respondent's year of birth" )

In [ ]:
def gen_cohot( series, size = 5 ):
    
    min1 = int( ( ( series.min() ) / size) ) * size
    max1 = int( np.ceil( ( series.max() ) / size) ) * size
    result = np.arange( min1, max1 + 1 , 5 )
    labels = np.ceil( ( result[ 1: ] + result[ :-1 ] ) / 2 ).astype( int )
    
    return result, labels

In [ ]:
gen_cohot( df.V010, size = 10 )

In [ ]:
cohorts, labels = gen_cohot( df.V010 )

In [ ]:
df[ 'birth_cohort' ] = pd.cut( df.V010, bins  = cohorts, 
                              include_lowest = True, labels = labels )

In [ ]:
df

In [ ]:
# Plot ploints
fig = plt.figure( figsize = ( 10, 9 ) )

sns.pointplot( x = "birth_cohort", y = "V201", 
               data = df, ci = 'sd')

# x label
plt.xlabel( "Cohort" )
plt.ylabel( "Total children ever born" )

In [ ]:
# Plot ploints
fig = plt.figure( figsize = ( 10, 9 ) )

sns.pointplot( x = "birth_cohort", y = "V201", 
               data = df, ci = 'sd', hue = 'urban', 
             dodge = 0.2, 
              linestyles = [ "-", "--" ] )

# x label
plt.xlabel( "Cohort" )
plt.ylabel( "Total children ever born" )

# Change legend title
plt.legend( title =  "Type of place\n of residence" )

## First sex relationship

In [ ]:
df.V525.unique()

In [ ]:
df[ 'first_sex' ] = df.V525.copy()

In [ ]:
df.first_sex.replace( 'Not had intercourse', np.nan, inplace = True )
df.first_sex.replace( 'No ha tenido relación sexual', np.nan, inplace = True )
df.first_sex.replace( 'Nunca tuvo relaciones sexuales', np.nan, inplace = True )
df.first_sex.replace( 'En la primera unión', 'At first union', inplace = True )
df.first_sex.replace( 'En la primera union', 'At first union', inplace = True )

In [ ]:
# Filter values in variables

filter1 = df.first_sex == 'At first union'
df.loc[ filter1 , 'first_sex' ] = df.loc[ filter1 , 'V511' ]

In [ ]:
df.first_sex = df.first_sex.astype( float )

In [ ]:
df.first_sex

Celaning region variable

In [ ]:
sns.displot( data = df, x = 'first_sex', 
            kind = 'kde', col = 'V024', 
            col_wrap = 3, 
            facet_kws = {'sharex' : False, 
                        'sharey' : False }, 
            hue = 'urban'
           )

In [ ]:
df[ 'dpto' ] = df.V024.copy()

In [ ]:
df.dpto = df.dpto.str.lstrip() \
            .str.replace( " ", "_" ) \
            .str.lower() \
            .apply( lambda x :  unidecode.unidecode( x ) )

In [ ]:
sns.displot( data = df, x = 'first_sex', 
            kind = 'kde', col = 'dpto', 
            col_wrap = 3, 
            facet_kws = {'sharex' : False, 
                        'sharey' : False }, 
            hue = 'urban'
           )

## Total children ever born by cohort and wife and husband education

In [ ]:
df[ 'h_educ_yr' ] = df.V715.copy()

df.h_educ_yr.replace( 'No sabe', np.nan, inplace = True )
df.h_educ_yr.replace( "Don't know", np.nan, inplace = True )

In [ ]:
df.loc[ ( df.V133 <= 12 ), 'w_educ' ] = 'Low'
df.loc[ ( df.V133 > 12), 'w_educ' ] = 'High'

df.loc[ ( df.h_educ_yr <= 12 ), 'h_educ_str' ] = 'Low'
df.loc[ ( df.h_educ_yr > 12), 'h_educ_str' ] = 'High'

In [ ]:
# Plot ploints
fig = plt.figure( figsize = ( 10, 9 ) )

sns.pointplot( x = "birth_cohort", y = "V201", 
               data = df, ci = 'sd', hue = 'w_educ', 
             dodge = 0.2, 
              linestyles = [ "-", "--" ] )

# x label
plt.xlabel( "Cohort" )
plt.ylabel( "Total children ever born" )

# Change legend title
plt.legend( title =  "Wife Education" )

In [ ]:
# Plot ploints
fig = plt.figure( figsize = ( 10, 9 ) )

sns.pointplot( x = "birth_cohort", y = "V201", 
               data = df, ci = 'sd', hue = 'h_educ_str', 
             dodge = 0.2, 
              linestyles = [ "-", "--" ] )

# x label
plt.xlabel( "Cohort" )
plt.ylabel( "Total children ever born" )

# Change legend title
plt.legend( title =  "Husband Education" )